#### `Time.now`是一个间接输入。首先我们引用了`Time`类，然后把`#now`消息发送给`Time`。我们想要的其实是方法`now`的返回值，而不是`Time`常量本身。任何时候为了得到方法返回值，而向其他对象发送消息，其实就是在使用间接输入。

间接输入层次越深，代码的耦合程度就越高

In [1]:
Place = Struct.new(:index, :name, :prize)

Place

In [2]:
first = Place.new(0, "first", "Peasant's Quest game")

#<struct Place index=0, name="first", prize="Peasant's Quest game">

In [3]:
second = Place.new(1, "second", "Limozeen Album")

#<struct Place index=1, name="second", prize="Limozeen Album">

In [4]:
thrid = Place.new(2, "third", "butter-da")

#<struct Place index=2, name="third", prize="butter-da">

In [5]:
winners = ["a", "b"]

["a", "b"]

In [6]:
[first, second, thrid].each do |place|
  p "In #{place.name} place, #{winners[place.index]}"
  p "you wind: #{place.prize}"
end

"In first place, a"
"you wind: Peasant's Quest game"
"In second place, b"
"you wind: Limozeen Album"
"In third place, "
"you wind: butter-da"


[#<struct Place index=0, name="first", prize="Peasant's Quest game">, #<struct Place index=1, name="second", prize="Limozeen Album">, #<struct Place index=2, name="third", prize="butter-da">]

`#to_s`就是一个显式转换方法。显式转换一般用于这样的情形：源类型和目标类型很大程度上不相关或者毫无关联

与之对应，`#to_str`就是一个隐式类型转换方法。隐式类型转换适用于元类型和目标类型很相近的情形

In [7]:
class EmacsConfigFile
  def initialize
    @filename = "#{ENV['HOME']}/.vimrc"
  end
  
  def to_path
    @filename
  end
end

:to_path

In [8]:
emacs_config = EmacsConfigFile.new

#<EmacsConfigFile:0x000000558f4cec50 @filename="/home/demouser/.vimrc">

In [9]:
File.open(emacs_config).lines.count

<main>: warning: IO#lines is deprecated; use #each_line instead


740

这是因为，`EmacsConfig`类定义了转换方法`#to_path`，而`File#open`又会在参数对象上调用`#to_path`方法，以便得到文件名字符串。这样一来，这个非字符串对象也能工作得很好。

下面的例子可以说明两者（隐式转化和显示转化的区别）。`Time`对象并非`String`，有太多方式可以将`Time`转化成`String`了。另外，这两种类型还是不想管的，因此`Time`只预定义了显式的类型转换方法`#to_s`，而没有定义隐式版的`#to_str`

In [10]:
now = Time.now
now.respond_to?(:to_s)
now.to_s
now.respond_to?(:to_str)

false

In [11]:
"hello, " + "world"

"hello, world"

如果`#to_str`所做的一切就是返回字符串自身，那么它看起来意义不大。`#to_str`的意义在于：许多`Ruby`方法都期望得到字符串输入，它们隐式地在输入对象上调用`#to_str`方法（这正是“隐式类型转换”这个术语的来源）

In [12]:
"I am a String".to_str

"I am a String"

In [13]:
class ArticleTitle
  def initialize(text)
    @text = text
  end
  
  def slug
    @text.strip.tr_s("A-Za-z0-0", "-").downcase
  end
  
  def to_str
    @text
  end
  
  def to_s
    to_str
  end
end

:to_s

In [14]:
title = ArticleTitle.new("A Modest Proposal")
"Today's future" +title

"Today's futureA Modest Proposal"

你可能还记得我说过Ruby核心类从来不会自动调用显式类型转换方法，但仍有例外，有一种情况下，Ruby语言却会自动调用显式类型转换方法。

In [15]:
"The time is now: #{Time.now}"

"The time is now: 2019-01-29 14:55:05 +0000"

In [16]:
PHONE_EXTENSIONS=["Operator", "Sales", "Customer Service"]

def dial_extension(dialed_number)
  dialed_number = dialed_number.to_i
  extension = PHONE_EXTENSIONS[dialed_number]
  puts "Please hold while you are connected to #{extension}"
end

dial_extension(nil)

Please hold while you are connected to Operator


如果只关心得到预期输入类型而不深究其源类型，则可以使用显式类型转化方法。

In [17]:
def set_centrifuge_speed(new_rpm)
  new_rpm = new_rpm.to_int
  puts "Adjusting centrifuge to #{new_rpm} RPM"
end

:set_centrifuge_speed

In [18]:
bad_input = nil
set_centrifuge_speed(bad_input)

NoMethodError: undefined method `to_int' for nil:NilClass

In [19]:
aaa = nil
"fe#{aaa}"

"fe"

某种情况下，如果方法的参数不符合预期类型或者与预期不接近，就会为程序带来隐患，这时最好用隐式类型转换

这里的隐式类型转换起到了“门卫”的作用：用以确保方法接受到的参数要么是`Integer`，要么可以很好地被转化为`Integer`。当参数不满足条件时，`NoMethodError`清楚地告诉了代码调用者调用非法。

In [20]:
def draw_line(start, endpoint)
  start = start.to_coords if start.respond_to?(:to_coords)
  start = start.to_ary
end

:draw_line

In [21]:
class Point
  attr_reader :x, :y, :name
  def initialize(x, y, name = nil)
    @x, @y, @name = x, y, name
  end
  
  def to_coords
    [x, y]
  end
end

:to_coords

In [22]:
start = Point.new(23,37)
endpoint = [45, 89]

draw_line(start, endpoint)

[23, 37]

方法就可以提供这样的接口：既可以接受普通输入，还能向前兼容那些更具语义的输入（如Point）

In [23]:
require 'forwardable'

class RecordCollection
  attr_accessor :records
  extend Forwardable
  def_delegator :@records, :[], :record_number
end

:record_number

In [24]:
r = RecordCollection.new
r.records = [4,5,6]
r.record_number(0)  # => 4

4

For example, say you have a class RecordCollection which contains an array @records. You could provide the lookup method record_number(), which simpley calls [] on the `records` array, like this

我们不能保证所有输入都是`Meter`类型而非Integer类型。我们也想支持那些能够用英尺表示，且能在运算中混合使用的度量单位。我们得想办法隐式地将`Meter`转换为`Feet`，以及将`Feet`转换为`Meter`

In [25]:
class Meters
  def to_meters
  end
end

class Feet
  def to_meters
  end
end

:to_meters

现在汇报还把变化，再也不用担心单位混淆了。因为我们确信任何不支持`to_meters`协议的对象都将触发`NoMethodError`异常

虽然`Integer()`可转换更多参数类型，但是对待字符串参数，它却比`#to_i`还挑剔

In [26]:
"ham sandwich".to_i

0

无论输入的原有类型是什么，你都渴望将其转换成特定原生类型。例如，无论输入的是`Float`类型的数字还是`nil`，甚至是十六进制字符串，只要可行，就将其转换成`Integer`类型

Ruby的类型转换方法可不止前面章节谈到的那些`#to_*`方法，Ruby核心模块还提供了一系列命名独特的强制类型转换方法，如`Array(), Float(), String(), Integer(),Rational()和Complex()`。通常，相对于对应`#to_*`版本的转换方法，它们可以转换更多的输入类型。

部分标准库也提供了强制类型转换方法

In [27]:
require 'pathname'
require 'uri'

path = Pathname.new("/etc/hosts")
p Pathname(path)
p Pathname("/etc/hosts")

uri_str = "http://example.org"
uri = URI.parse(uri_str)
p URI(uri_str)
p URI(uri)

#<Pathname:/etc/hosts>
#<Pathname:/etc/hosts>
#<URI::HTTP http://example.org>
#<URI::HTTP http://example.org>


#<URI::HTTP http://example.org>

下面的方法接受文件名为参数，报告文件大小。这里的文件名可以是`Pathname`对象，也可以是其他任何可悲转换为`Pathname`的对象

In [28]:
def file_size(filename)
  filename = Pathname(filename)
  filename.size
end

:file_size

In [29]:
file_size(Pathname.new("/etc/hosts"))
file_size("/etc/hosts")

1223

`Kernel#Array`会不遗余力地将输入转换为数组

In [30]:
Array("foo")

["foo"]

In [31]:
Array([1,2,3])

[1, 2, 3]

In [32]:
Array([])

[]

In [33]:
Array(nil)

[]

In [34]:
Array({:a =>  1, :b => 2})

[[:a, 1], [:b, 2]]

In [35]:
Array(1..5)

[1, 2, 3, 4, 5]

In [36]:
def log_reading(reading_or_readings)
  readings = Array(reading_or_readings)
  readings.each do |reading|
    puts "[READING] %3.2f" % reading.to_f
  end
end

:log_reading

In [37]:
log_reading(3.14)

[READING] 3.14


[3.14]

In [38]:
log_reading([])

[]

In [39]:
log_reading([87.5, 45.8765, 22])

[READING] 87.50
[READING] 45.88
[READING] 22.00


[87.5, 45.8765, 22]

In [40]:
log_reading(nil)

[]

一个对象提出的要求越少，就越容易被使用。若一个对象能为多种不同类型的的对象提供服务，那么将它作为一个辅助对象，在履行自身职责的时候，就不会向周围对象提出要求。

我们将采用Ruby核心类和标准库中强制类型转换方法的风格来定义一个名称特别的转换方法，它们目标类Point来命名

In [41]:
module Graphics
  module Conversions
    module_function
    def Point(*args)
      case args.first
      when Point then args.first
      when Array then Point.new(*args.first)
      when Integer then Point.new(*args)
      when String then
        Point.new(*args.first.split(':').map(&:to_i))
      else
        raise TypeError, "Cannot convert #{args.inspect} to Point"
      end
      end
      end
        Point = Struct.new(:x, :y) do
          def inspect
            "#{x}:#{y}"
          end
        end
      end

Graphics::Point

In [42]:
include Graphics
include Graphics::Conversions
Point(Point.new(2,3))
Point([9,7])
Point(3,5)
Point("8:10")

TypeError: Cannot convert [#<Point:0x000000558fc51900 @x=2, @y=3, @name=nil>] to Point

接着我们就可以在代码中使用该转换方法了，尤其是和外来输入打交道的时候。无论何处，当不能确定输入类型时候，我们要做的不是浪费时间来摸清各种可能性，而是简单地用`Point()`方法将其包起来。然后，既然知道只会和`Point`对象打交道，便可将注意力转移到手边的业务逻辑上来。

`module_function`做了两件事，首先，将紧跟其后的方法都标注成了私有方法；其次，让紧跟其后的方法成了低昂前模块的单例方法

通过将#Point标注成私有方法，我们就可以将其封装在内部，从而有别于那些共有的接口方法。举个例子，假设有一个`canvas`对象已经引入了`Conversations`模块，那么应该没必要在该对象之外再来调用`canvas.Point(1,2)`吧。

In [43]:
def Point(*args)
  case args.first
    when Integer then Point.new(*args)
    when String then Point.new(*(args.first.split(':').map(&:to_i))
    when ->(arg) {arg.respond_to?(:to_point))}
      args.first.to_point
    when ->(arg) {arg.respond_to?(:to_ary)}
      Point.new(*args.first.to_ary)
    else
      raise TypeError, "Cannot convert #{args.inspect} to Point"
    end
    end

SyntaxError: <main>:4: syntax error, unexpected keyword_when, expecting ')'
    when ->(arg) {arg.respond_to?(:to...
    ^~~~
<main>:4: syntax error, unexpected ')', expecting '}'
...g) {arg.respond_to?(:to_point))}
...                              ^
<main>:6: syntax error, unexpected keyword_when, expecting '}'
    when ->(arg) {arg.respond_to?(:to...
    ^~~~
<main>:8: syntax error, unexpected keyword_else, expecting '}'
    else
    ^~~~
<main>:10: syntax error, unexpected keyword_end, expecting '}'
    end
    ^~~

  1. 标准库`#to_ary`，匹配数组，以及不是数组但可以转换为数组的对象
  2. 我们库中的`#to_point`，匹配那些定义了将自己转换为Point的对象

现在我们为程序提供了两个扩展点。我们不再排斥类数组对象（就差从`Array`继承了），并且现在的客户端代码可以自定义`#to_point`方法了（而非原来那样强制输入类型必须为`Point`）。

In [44]:
even = ->(x) {(x % 2) == 0}

#<Proc:0x00000055904492e8@<main>:0 (lambda)>

In [45]:
even === 4

true

In [46]:
even === 9

false

`case`语句使用三等运算符`（#===）` 来判定分支是否匹配，而`Ruby`的`Proc`对象正好定义了三等运算符（`#call`的别名）

In [47]:
class TrafficLight
  def change_to(state)
    @state = state
  end
  
  def signal
    case @state
    when "stop" then turn_on_lamp(:red)
    when "caution"
      turn_on_lamp(:yellow)
      rignt_warning_bell
    when "proceed" then turn_on_lamp(:green)
    end
    end
  
  def next_state
    case @state
    when "stop" then "proceed"
    when "caution" then "stop"
    when "proceed" then "caution"
    end
    end
      
  def turn_on_lamp(color)
    puts "Turning on #{color} lamp"
  end
      
  def ring_warning_bell
    puts "Ring ring ring"
  end
end

:ring_warning_bell

In [48]:
light = TrafficLight.new
light.change_to("PROCEED")
light.signal
puts "Next state:#{light.next_state.inspect}"

light.change_to(:stop)
light.signal
puts "Next state:#{light.next_state.inspect}"

Next state:nil
Next state:nil


该`case`并无`else`分支，这就意味着如果`@state`不是`stop`、`caution`或者`proceed`其中之一时，该方法会悄然出错。

我们可以通过改进`#change_to`方法，让它防范非法输入。

In [49]:
def change_to(state)
  raise ArgementError unless ["stop", "proceed", "caution"].include?(state)
  @state = state
end

:change_to

如果分别用独立的对象来表示红绿灯状态，会不会好点呢

In [50]:
class TrafficLight
  State = Struct.new(:name) do
    def to_s
      name
    end
  end
  
  VALID_STATES = [
    STOP = State.new("stop"),
    CAUTION = State.new("caution"),
    PROCEED = State.new("proceed")
    ]
  
  def change_to(state)
    raise ArgumentError unless VALID_STATES.include?(state)
    @state = state
    end
    
  def signal
    case @state
    when STOP then turn_on_lamp(:red)
    when CAUTION
      turn_on_lamp(:yellow)
      ring_warning_bell
    when PROCEED then turn_on_lamp(:green)
    end
    end
  
  def next_state
    case @state
    when STOP then 'proceed'
    when CAUTION then 'stop'
    when PROCEED then 'caution'
    end
    end    
    end

:next_state

我们再次探讨`state`的定义。这次将其申明为子类，而不再是实例对象。

In [51]:
class TrafficLight
  class State
    def to_s
      name
    end
    
    def name
      self.class.name.split('::').last.downcase
    end
    
    def signal(traffic_light)
      traffic_light.turn_on_lamp(color.to_sym)
    end
    
  end
    class Stop < State
      def color; 'red'; end
      
      def next_state; Proceed.new; end
    end
    
    class Caution < State
      def color; 'yellow'; end
      def next_state;  Stop.new; end
    
    
    
    def signal(traffic_light)
      super
      traffic_light.ring_warning_bell
    end
  end
  
  class Proceed < State
    def color; 'green'; end
    
    def next_state; Caution.new; end
  end
end

:next_state

In [52]:
class TrafficLight
  
  def next_state
    @state.next_state
  end
  
  def signal
    @state.signal(self)
  end
  
end

:signal

不幸的是,`TrafficLight`使用`State`的便利性反而变更糟了

In [53]:
light = TrafficLight.new
light.change_to(TrafficLight::Caution.new)
light.signal

ArgumentError: ArgumentError

In [54]:
class TrafficLight
  def change_to(state)
    @state = State(state)
  end
  
  private
  
  def State(state)
    case state
    when State then state
    else self.class.const_get(state.to_s.capitalize).new
    end
    end
  end

:State

现在可以这样使用了

In [55]:
light = TrafficLight.new
light.change_to(:caution)
light.signal
puts "Next state is: #{light.next_state}"

Turning on yellow lamp
Ring ring ring
Next state is: stop


高效地使用面向对象语言的要点就是：让封装和多台替你分忧解劳。面对那些来自外界的字符串或`Symbol`，简单的确保它们是合法的，然后就可以撒手不管了，这看起来比较诱人。然而，细看之下，你可能发现明显的领域概念开始浮出水面。用类来反映这些概念，不仅能减少代码出错，还有助于加深我们对问题的理解，同事替身解决该问题的程序设计。这也意味着方法减少了花费在输入检查上的时间，我们就可以腾出更多尽力来关注业务逻辑本身。

In [56]:
class BenchmarkedLogger
  def initialize(sink=$stdout)
    @sink = sink
  end
  
  def info(message)
    start_time = Time.now
    yield
    duration = start_time - Time.now
    @sink << ("[%1.3f] %s\n" % [duration, message])
  end
end
  

:info

使用文件、标准输出、标准错误输出、网络嵌套字，或者内存数组做日志终端时候，该类可以很好地工作。之所以能和这些终端很好地工作，是因为他们都支持`<<`（追加操作符）。换句话说，所有这些终端类型都共享了相同的接口。

IRC logging bot如下

In [57]:
require 'cinch'

bot = Cinch::Bot.new do
  configure do |c|
    c.nick = "bm-logger"
    c.server = ENV["LOG_SERVER"]
    c.channels = [ENV["LOG_CHANNEL"]]
    c.verbose = true
  end
  on :log_info do |m, line|
    Channel(ENV["LOG_CHANNEL"]).msg(line)
  end
end

bot_thriead = Thread.new do
  bot.start
end

LoadError: cannot load such file -- cinch

要想将日志信息写进IRC频道，我们得出发自定义的`:log_info`事件

In [58]:
bot.handlers.dispatch(:log_info, nil, "Something happend")

NoMethodError: undefined method `handlers' for nil:NilClass

既然`#<<message`是一种简单易懂的日志输出方式，而且支持大多数日志终端，那么不妨使用`IRC Bot`的时候就用适配器修饰它，让它支持通用的接口，避免`#info`和其他方法被类型检查干扰

In [59]:
class BenchmarkedLogger
  class IrcBotSink
    def initialize
      @bot = bot
    end
    
    def <<(message)
      @bot.handlers.dispatch(:log_info, nil, message)
    end
  end
  
  def initialize(sink)
    @sink = case sink
    when Cinch::Bot then IrcBotSink.new(sink)
    else sink
    end
    end
  end
  

:initialize

In [60]:
require 'cinch'
require 'delegate'

class BenchmarkedLogger
  class IrcBotSink < DelegateClasss(Cinch::Bot)
    def <<(message)
      handlers.dispatch(:log_info, nil, message)
    end
    
    def initialize(sink)
      @sink = case sink
      when Cinch::Bot then IrcBotSink.new(sink)
      else sink
      end
    end
    end
end


LoadError: cannot load such file -- cinch

我们将用Ruby标准库中的`DelegateClass`类作为透明适配器的基础。它将生成这样一个类：所有调用该类的方法都将传给底层的对象。所以，在`IrcBotSink`中调用`#handlers`方法，实际上调用的是内部`Cinch::Bot`的`#handlers`。

实际接受方法调用的底层对象`Cinch::Bot`是通过构造方法传入的。而`DelegateClass`恰好提供这样的构造方法。我们要做的只是为适配器对象提供要增加的方法。

为了实现这一切，我们还得做些调整：将当前代码中所有引用`Cinch::Bot`的地方统统换成`IrcBotSink`。这只需要搜搜替换就能完成。

乍看之下，这并无特别之处。但是，使用透明适配器对象替换`Cinch::Bot`对象之后，我们离统一日志终端接口又近了一步。我们逐一将蹩脚的`switch`语句都替换成统一的接口，同时又不比担心破坏尚未来的及跟新的遗留方法。

让程序进校抛出明显的错误，总好过就不正常、将来突然出现令人困惑的一场问题

In [61]:
require 'date'

class Employee
  attr_accessor :name
  attr_accessor :hire_date
  
  def initialize(name, hire_date)
    @name = name
    @hire_date = hire_date
  end
  
  def due_for_tie_pin?
    raise "Missing hire date!" unless hire_date
    ((Date.today - hire_date) / 365).to_i >= 10
  end
  
  def covered_by_pension_plan?
    ((hire_date && hire_date.year) || 2000) < 2000
  end
  def bio
    if hire_date
      "#{name} has been a Yoyodyne employee sinece #{hire_date.year}"
    else
      "#{name} is a proud Yoyodyne employee"
    end
  end
end

:bio

这是一个必须做“边界检查”的例子。既然没有一种很好地方式来应对入职日期确实的情况，不如简单地坚持入职日期必须有合法值。这样就可以让那些`nil`无处藏身了。

In [62]:
require 'date'

class Employee
  attr_accessor :name
  attr_reader :hire_date
  def initialize(name, hire_date)
    @name = name
  end
  
  def hire_date=(new_hire_date)
    raise TypeError, "Invalid hire date" unless
    new_hire_date.is_a?
    @hire_date = new_hire_date
  end
end

:hire_date=

**遵循先决条件应该是方法调用者的职责，也就是说，调用者用用不应该拿违反先决条件的输入法去调用方法**。但`Ruby`并不支持契约设计，所以我们将先决条件放到待保护方法开始部分。

先决条件身兼双重职责。首先，它们阻止非法输入，从而避免非法输入造成方法出现难以预料的行为；其次，它们位处方法的开头部分，作为“可执行文档”表明输入要求。读代码时，第一眼看到的便是说明哪些输入不合法的先决条件。

有些方法的输入时不可接受的。有时它们只在某个地方引起错误，这并无大碍；但在另一些条件下，非法输入则会危害系统，甚至是业务。更可怕的是，开发人员对非法输入的担心，会影响系统的统一性。在入口处及时拒绝无效输入，可以让程序变得更简装，简化内部逻辑，同时为其他开发人员提供“可执行文档”

假设有这样一个方法，用于包装useradd，它用`Hash`来接收所得中属性输入

In [63]:
def add_user(attributes)
  login = attributes[:login]
  unless login
    raise ArgumentError, 'Login must be supplied'
  end
  password = attributes[:password]
  unless password
    raise ArgumentError, 'Password (or false) must be supplied'
  end
  
  icoomand = %[useradd]
  
  if attributes[:home]
    command << '--home' << attributes[:home]
  end
  if attributes[:shell]
    command << '--shell' << attributes[:shell]
  end
  
  if attributes[:dry_run]
    puts command.join(" ")
  else
    system *command
  end
end

:add_user

In [64]:
def test
  value = yield
  if value
    "truthy (#{value.inspect})"
  else
    "falsey (#{value.inspect})"
  end
rescue =>error
  "error (#{error.class})"
end

:test

In [65]:
h = {:a =>123, :b =>false, :c =>nil}

{:a=>123, :b=>false, :c=>nil}

In [66]:
test {h[:a]}

"truthy (123)"

In [67]:
test {h[:b]}

"falsey (false)"

In [68]:
test {h[:c]}

"falsey (nil)"

In [69]:
test {h[:x]}

"falsey (nil)"

In [70]:
p h[:x]

nil


In [71]:
test {h.fetch(:a)}

"truthy (123)"

In [72]:
test {h.fetch(:b)}

"falsey (false)"

In [73]:
test {h.fetch(:c)}

"falsey (nil)"

In [74]:
test {h.fetch(:x)}

"error (KeyError)"

## Fetch写法

In [75]:
def add_user(attributes)
  login = attributes.fetch(:login)
  password = attributes.fetch(:password)
  
  command = %w[useradd]
  if attributes[:home]
    command << '--home' << attributes[:home]
  end
  if attributes[:shell]
    command << '--shell' << attributes[:shell]
  end
  
  if passward == false
    command << '--disabled-login'
  else
    command << '--password' << password
  end
  
  command << login
  
  if attrubtes[:dry_run]
    puts command.join(" ")
  else
    system *command
  end
end

:add_user

In [76]:
attributes = {}

password = attributes.fetch(:password) do
  raise KeyError, "Password (or false) must be supplied"
end


KeyError: Password (or false) must be supplied

In [77]:
require 'nokogiri'
require 'net/http'
require 'tmpdir'
require 'logger'

def emergency_kittens(options = {})
  logger = options[:logger] || default_logger
  uri = URI("http://api.flickr.com/services/feeds/photos_public.gne?tags=kittens")
  logger.info "Finding cuteness"
  body = Net::HTTP.get_response(uri).body
  feed = Nokogiri::XML(body)
  image_url = feed.css('link[rel=enclosure]').to_a.sample['href']
  image_uri = URI(image_url)
  logger.info "Downloading cuteness"
  open(File.join(Dir.tmpdir, File.basename(image_uri.path)), 'w') do |f|
    data = Net::HTTP.get_response(URI(image_url)).body
    f.write(data)
    logger.info "Cuteness written to #{f.path}"
    return f.path
  end
end

def default_logger
  l = Logger.new($stdout)
  l.formatter = -> (severity, datetime, progname, msg){
    "#{severity} -- #{msg}\n"
    }
  l
end

:default_logger

In [78]:
emergency_kittens

INFO -- Finding cuteness
INFO -- Downloading cuteness
INFO -- Cuteness written to /tmp/33028102838_d6f2817cc5_b.jpg


"/tmp/33028102838_d6f2817cc5_b.jpg"

In [79]:
simple_logger = Logger.new($stdout)
simple_logger.formatter = -> (_, _, _, message) {
  "#{message}\n"
  }

emergency_kittens(logger: simple_logger)

Finding cuteness
Cuteness written to /tmp/45999975745_9ce97bfe35_b.jpg


"/tmp/45999975745_9ce97bfe35_b.jpg"

In [80]:
options = {}
options[:logger] = false
logger = options[:logger] || Logger.new($stdout)
if logger == false
  logger = Logger.new('/dev/null')
end


In [81]:
emergency_kittens(logger: false)

INFO -- Finding cuteness
INFO -- Downloading cuteness
INFO -- Cuteness written to /tmp/46856059512_9ae0535988_b.jpg


"/tmp/46856059512_9ae0535988_b.jpg"

由于`false`表示假，故`logger = options[:logger] ||Logger.new($stdout)`会生成`$stdout`日志，而非将其设置为`false`。所以，原打算在日志属性为`false`时将其替换为`/dev/null`的代码从未触发过。

In [82]:
logger = options.fetch(:logger) {Logger.new($stdout)}
if logger == false
  logger = Logger.new('/dev/null')
end

#<Logger:0x000000558f75e1f0 @level=0, @progname=nil, @default_formatter=#<Logger::Formatter:0x000000558f75e1a0 @datetime_format=nil>, @formatter=nil, @logdev=#<Logger::LogDevice:0x000000558f75e150 @shift_period_suffix="%Y%m%d", @shift_size=1048576, @shift_age=0, @filename="/dev/null", @dev=#<File:/dev/null>, @mon_owner=nil, @mon_count=0, @mon_mutex=#<Thread::Mutex:0x000000558f75e100>>>

我们可以用`Hash#fetch`和代码块来根据条件提供默认日志对象，从而修复该`bug`。因为`#fetch`紧急当特定键确实时才执行并返回代码块中的内容。所以显示地设置`:logger`为false不会被覆盖

代码块形式的`#fetch`更明确地表示了“这便是默认值”

在某些库中，相同默认值会一遍一遍的重复。这种情况下，可以很方便的将默认值设置为`Proc`，然后将其作用每个`#fetch`的默认值

为什么不用

In [83]:
logger = options.fetch(:logger, Logger.new($stdout))

false

default作为参数传给`#fetch`时候，无论是否需要，它总是会被执行。现在“昂贵”代码每次都会被执行，及时所求值已然存在。这还不仅仅是性能消耗的问题，如果`default`方法中的代码有副作用呢（如向数据库插入数据）？

## 用断言验证假设

方法接受的输入来自外部系统（如银行交易数据），这类输入可能疏于文档，而且极不稳定

在初次用到输入格式假设的地方，便用断言来加以假设。断言失败可以加深你对输入的理解，并且在输入格式发生变化时及时发出警告

In [84]:
class Account
  def refresh_transactions
    transactions = bank.read_transactions(account_number)
    transactions.is_a?(Array) or raise TypeError, "transactions is not an Array"
  end
end

:refresh_transactions

通常，我们将对象类型检查视为代码坏味道（code smell），但是，就现在的情况而言，还是努力抓住每根救命稻草为好。在代码边界处，我们对输入的数据的把握越明确，后期协作就越放心。

我们更加倾向于更加明确地验证假设。用`Hash#fetch`作断言，代码如下

In [85]:
transactions.echo do |transaction|
  amount = transaction.fetch("amount")
end 


NameError: undefined local variable or method `transactions' for main:Object

相比于`String#to_f`,`Kernel#Float`更加严格。

In [86]:
Float("$1.23")

ArgumentError: invalid value for Float(): "$1.23"

最终地代码充满断言，但这是必须地

In [87]:
class Account
  def refresh_transactions
    transactions = bank.read_transactions(account_number)
    transactions.is_a(Array) or raise TypeError, "transcations is not a array"
    
    transactions.each do |transaction|
      amount  = transaction.fetch("amount")
      amount_cents = (Float(amount) * 100).to_i
      cache_transaction(:amount => amount_cents)
    end
  end
end

:refresh_transactions

该代码清除地表明了自己地假设。它包含了大量我们对阿文i不API地理解信息，它显示地建立了一系列可以放心使用地函数，一旦输入与参数不服，就会及时报错，同时带有明确地异常信息。

及时报错而非纵容无效输入污染整个系统，可以减少类型检查和强制类型转换地使用。这些代码不仅现在验证了我们地假设，而且建立了一套预警机制，以防未来某天第三方API意外地变化。

通过在入口处利用断言表明假设，我们永久地记录了对输入数据地假设。这样我们便保护了内部代码免受外部输入地干扰。于此同时，我们还建立了一套预警机制，一旦我们对外部系统地假设国企，就及时警告我们

## 用卫语句来处理特殊场景

如果使用`if-else`结构，那么其实你赋予了`if`和`else`相同地重要性，并且暗示读者：这两个分支可能性相同且同等重要。与之相反，卫语句则强调：这是特殊情况，如果发生了，稍做处理即可

In [88]:
def log_reading(reading_or_readings)
  unless @quiet
    readings = Array(reading_or_readings)
    readings.each do |reading|
      puts "[READING] %3.2f" % reading.to_f
    end
  end
end

:log_reading

用卫语句地写法

In [89]:
def log_reading(reading_or_readings)
  return if @quiet
  readings = Array(reading_or_readings)
  readings.each do |reading|
    puts "[READING] %3.2f" % reading.to_f
  end
end

:log_reading

某些特许情况需要尽早处理，但有些特殊请款又没必要让其凌驾于整个方法逻辑之上。用卫语句快速处理特殊情况，可使方法体免于特殊处理地干扰。

In [91]:
def current_user
  if session[:user_id]
    User.find(session[:user_id])
  end
end

:current_user

下面便是`current_user`的典型应用场景：先判断其返回值，若非空，则使用；否则，插入占位符：

In [92]:
def greeting
  "Hello," + current_user ? current_user.name : "Anonymous" +", how are you today?"
end

:greeting

上面的例子都有一个特征：不确定`#current_user`到底是返回用户对象，还是`nil`。因此，对`nil`的检测一次又一次肆意的重复着。

我们不再将匿名用户表示成`nil`，而是用一个类来表示它，取名为`GuestUser`。

In [93]:
class GuestUser
  def initialize
    @session = session
  end
end

:initialize

下面改进`#current_user`方法，在它没有`:user_id`时返回`GuestUser`对象。

In [94]:
def current_user
  if session[:user_id]
    User.find(session[:user_id])
  else
    GuestUser.new(session)
  end
end

:current_user

同时给GuestUser加上对应的`#name`属性，代码如下

In [96]:
class GuestUser
  def name
    "Anonymous"
  end
end

:name

这样子，`greeting`方法的代码变得漂亮多了

In [97]:
def greeting
  "Hello, #{current_user}, how are you today"
end

:greeting

针对根据用户登录状态来切换“登录”和“注销”按钮的情况，我们无法移除条件从句，不过可以给`User`和`GuestUser`都加上`#authenticated?`断言方法

In [98]:
class User
  def authenticaeted?
    true
  end
end

:authenticaeted?

In [100]:
class GuestUser
  def authenticated?
    false
  end
end

:authenticated?

使用断言之后，条件从句的意图清晰多了

In [101]:
if current_user.anthenticated?
  render_logout_button
else
  render_login_button
end

NameError: undefined local variable or method `session' for main:Object

价差用户是否有`admin`权限

In [103]:
class GuestUser
  def has_role?(role)
    false
  end
end

:has_role?

In [104]:
if current_user.has_role?(:admin)
  render_admin_panel
end

NameError: undefined local variable or method `session' for main:Object

针对根据用户的登陆状态来调整`@listing`的情况，我们给`GuestUser`加上`#visiable_listings`方法，让其简单地返回公共资源列表

In [105]:
class GuestUser
  def visiable_listings
    Listing.publicly_visiable
  end
end

:visiable_listings

先前地代码被简化成了单行形式，如下

```ruby
@listings = current_user.visible_listings
```

为了让`GuestUser`完全被作为普通用户对待，我们给它加上不做事地属性`setter`方法。

In [106]:
class GuestUser
  def last_seen_online=(time)
  end
end

:last_seen_online=

这又简化了另一个条件语句

In [107]:
current_user.last_seen_online = Time.now

NameError: undefined local variable or method `session' for main:Object

为了让尚未登陆地游客也加上了购物车，我们让`GuestUser`地`cart`属性返回先前的`SessionCart`对象

调整后，往购物车加商品的代码也变成了单行模式

这里我们提取出一个适用于多个方法的通用角色——用户，同时意识到用户登录并不意味着没有用户，而是表示我们面临的是一种特殊的匿名用户。这一发现促我们从方法构建层面回到面向对象设计中去。通过着眼于方法层面的代码实现，我们最终得到了更好的领域对象设计。

我们清楚，要打交道的是用户（无论登录是否），并且不想在代码清晰度和优雅度方面拖鞋。但是，我们还没有准备好将整个代码库从`nil`检测迁移到新的`GuestUser`上去，因此决定只在此处使用新类。下面便是引入`GuestUser`后的代码

In [109]:
def greeting
  user = current_user || GuestUser.new(session)
  "Hello, #{user.name}, how are you today"
end

:greeting

 `GuestUser`现在有了落脚点，而`#greeting`则变成了重新设计的实验基地。如果满意`GuestUser`在这里的表现，则可推广到其他类似的代码中区。最终，将`GuestUser`对象的创建零散地分布在各处

若多处都考虑同一特殊场景，则将导致`nil`检测不断重复。这些没完没了的对象存在性检查将代码弄得一团糟，并且非常容易因漏掉某个`nil`检测而引入`bug`。

使用特例对象，便将普通场景和特殊场景的区别集中到了同一的地方，然后多态会确保自会有正确的代码会被执行。这样一来，最终产品将更清晰和简要，同时职责划分的也更合理。

通过输入是否为`nil`来控制状态变迁，可以看成一种警告信号：特例对象或许是一种更好的解决方案。为了避免条件语句，我们可引入类来表示特殊场景，然后在当前方法中做试点。一旦创建好了特例类，并且确信他可优化代码中的列成和组织结构，便可重构更多的方法，让其使用特例对象而非条件语句。

In [110]:
class FFMPEG
  def record_screen(filename)
    source_options = %W[-f x11grab]
    recording_options = %W[-s #{@width}x#{@height} -i 0:0+#{@x}#{@y} -r 30]
    
    ffmpeg_flags = source_options + recording_options + misc_options + output_options
    
    if @logger
      @logger.info "Executing: ffmpeg #{ffmpeg_flags.join(' ')}"
    end
    system('ffmpeg', *ffmpeg_flags)
  end
end

:record_screen

如果每次输出日志都检测`logger`的存在性，将导致代码混乱不堪。更糟糕的是，在多次进行`logger`存在性检测后可能就懈怠了，因为每次都得为此付出额外的精力。

它们都有自己的实现，但共享了相同的接口。它们都实现了`#debug, #info, #info, #warn, #error, #fatal`这些不同级别的日志输出方式。无论底层是什么，我们得代码均可无差异地与其交互。

从另一个角度来看，`nil`也算是`logger`的一种特例。意识到这一点后，我们想到用于特殊场景的模式或许也可以应用于此

In [111]:
class NullLogger
  def debug(*) end
  def info(*) end
  def warn(*) end
  def error(*) end
  def fatal(*) end
end

:fatal

NullLogger类实现了`logger`的所有接口，但不是将日志输出到设备上，而是简单地接受参数却不做任何处理。我们可以将`NullLogger`对象用于`FFMPEG`中`@logger`地默认值

In [112]:
class FFMPEG
  def initialize(logger=NullLogger.new) end
end


:initialize

现在，输入命令日志哪一行不再需要`if`语句了

在当前的特殊场景下，对`logger`确实的特殊处理便是不做任何事。这里，我们需要一个不做任何事的特例对象，因它及其通用，因此有它自己的名称；空对象模式。“空对象实现了原型对象相关接口，只是这些方法的实现是不做任何事或返回恰当的默认值而已”

### 通用空对象

In [114]:
class NullObject < BasicObject
  def method_missing(*)
  end
  
  def respond_to?(name)
    true
  end
end

:respond_to?

假设有一个方法用于创建`HTTP`请求，同时收集一些参数

In [115]:
def send_request(http, request, metrics)
  metrics.requests.attempted += 1
  response = http.request(request)
  metrics.requests.successful += 1
  metrics.responses.codes[response.code] += 1
  response
rescue SocketError
  metrics.errors.socket += 1
  raise
rescue IOError
  metrics.errors.io += 1
  raise
raise HTTPError
end

:send_request

这段代码极度依赖`metrics`对象，且嵌套层次颇深。例如，哪行统计套接字错误的代码就涉及4次消息发送

1. `metrics`接收`erros`消息
2. 前面返回的对象`errors`接收`.socket`消息
3. 前面返回的对象`socket`接收`.+`消息
4. 第二步返回对象`socket`接收`.socket=`

我们并非所有时候都想手机这些参数，或许仅在打开“诊断模式”时才需收集。那其他是偶怎么办呢？不妨尝试用`Null Object`去替换`metric`对象

In [117]:
def send_request(http, request, metrics = NullObject.new)
end

:send_request

变异版的`NullObject`（空对象黑洞）

In [118]:
class NullObject
  def method_missing(*)
    self
  end
end

:method_missing

我们再引入“空对象黑洞”时候，我们要格外小心。具体来说，我们要确保“空对象黑洞”永远不会泄露到对象库或兑现股社区外面去。为了方便起见，我们定义了一个特殊的转换方法。

In [119]:
def Actual(object)
  case object
    when NullObject then nil
    else object
    end
  end

:Actual

In [120]:
Actual(User.new)

#<User:0x000000558fc3a9a8>

In [121]:
Actual(nil)

In [122]:
Actual(NullObject.new)

任何时候，若方法为公共`API`，且其返回值还可能被用到，都使用`Actual`来过滤一遍，以防空对象泄露

无论我们怎么努力尝试让空对象表现接近Ruby原生的`nil`，包括在`false`的表现，最终发现的苦苦追寻的是一条死路。

In [131]:
def render_member
  html = ""
  html << "<img class='photo' src='#{member.avatar_url}' />"
  location = Geolocation.locate(member.address)
  html << "<img class='map' src='#{location.map_url} />"
  html << "</div>"
end

:render_member

显示成员地图并非必须的，而是锦上添花。即使个别成员地图不能正常显示，我们还是希望继续渲染其他成员的信息。有一下集中方式可以解决这个问题

In [133]:
def render_member
  html = ""
  html << "<img class='photo' src='#{member.avatar_url}' />"
  location = Geolocation.locate(member.address)
  html << "<img class='map' src='#{location.map_url} />"
  html << "</div>"
  begin
    location = Geolocatron.locate(member.address)
    html << " <img class='map' src='#{location.map_url}"
  rescue NoMethodError
  end
  html << "</div>"
end

:render_member

另一种方法是在`location`对象使用前进行存在性检查

In [134]:
def render_member
  html = ""
  html << "<img class='photo' src='#{member.avatar_url}' />"
  location = Geolocation.locate(member.address)
  html << "<img class='map' src='#{location.map_url} />"
  html << "</div>"
  if location
    html <<" <img class='map' src='#{location.map_url} />"
  end
  html << "</div>"
end

:render_member

就程序流畅度而言，上面两种方案都是有问题。成员地图是非必须的，属于`#render_member`方法的次要任务。但是，无论是`begin/rescue/end`还是`if`从句都把大量精力放在`location`缺失的可能性的。这样的代码正如“爱哭的小孩有糖吃”一样：它可能并非方法最重要的部分，但却得到诸多特殊关照，以至于其他部分都无足轻重了

如果不在对`location`的缺失加以特殊关照，而是为期提供备用数据会怎么样呢？不妨用组内成员所在城市区域的`location`来作为备用数据。既然到了这一步，顺便将`location`查找的代码移动方法开始部分，这样便不会破坏渲染`HTML`文本的节奏感了

In [136]:
def render_member(member, group)
  location = Geolocatron.locate(member.address) || group.city_location
  html = ""
  html << "<div class='vcard'>"
...
  end

SyntaxError: <main>:4: syntax error, unexpected ..., expecting keyword_end

`nil`因其普遍存在性，在出现异常时传达的语义极少，甚至根本没有。那我们如何才能更好的表达异常呢?

让我们回到`#list_widgets`方法，不过这次求助老朋友`Hash#fetch`来为`credentials`提供默认值

In [138]:
def list_widgets(options = {})
  credentials = options.fetch(:credentials) {:credentials_not_set}
  page_size = options.fetch(:page_size)
  page = options.fetch(:page) {1}
  if page_size >20
    user = credentials.fetch(:user)
    password = credentials.fetch(:password)
    url = "https://https://..."
  else
    url = "http://www.example.com/widgets"
  end
  puts "Contacting #{url}"
end

:list_widgets

这次调用`list_widgets(page_size: 50)`

-：7 in`list_widgets`:undefined method `fetch` for `:credentials_not_set:Symbol`

虽然还是`NoMethodError`，但这次`NoMethodError`发生在`:credentials_not_set`上，这有俩明显好处

1. 错误信息按时了我们哪里错了，看起来使我们未设置认证信息造成的
2. 即使我们还是没能从中发生错误原因，`:credentials_not_set`也提供了查找的线索，我们还可以轻易的将错误定位到这一行

有多种方式可以让必要输入项缺失引发的错误变得清晰起来。如果认证信息缺失是一个普遍存在的问题，且许多用户将使用该库，我们可能会显示检测，且出错抛出带语义的异常。有些场景下，我们甚至会使用特例对象作为默认值。但是，带雨衣的占位符是其中最物超所值的一个：紧需一行代码的修改，便可大幅提升后续错误信息的语义质量。

假设有几个不同的类都和地图上的点打交道。一个点由x和y坐标组成，故自然而然地每个与点打交道的方法都接收X和Y坐标做参数，包括那些画点、画线的方法。

In [139]:
class Map
  def draw_point(x, y)
  end
  def draw_line(x1, y1, x2, y2)
  end
end

:draw_line

In [140]:
class MapStore
  def write_point(x, y)
    point_hash = {x: x, y: y}
  end
end

:write_point

显然，X和Y坐标总会成对出现，因而很适合将其封装到类中。这里我们使用`Struct`便轻松构造了一个类

In [141]:
Point = Struct.new(:x, :y)

<main>: warning: already initialized constant Point


Point

可以让`Point`去花他们自己

In [142]:
Point = Struct.new(:x, :y) do
  def draw_on(map)
  end
end

class Map
  def draw_point(point)
    point.draw_on(self)
  end
  
  def draw_line(point1, point2)
    point1.draw_on(self)
    point2.draw_on(self)
  end
end

<main>: warning: already initialized constant Point


:draw_line

重构已经卓有成效，原因如下：精简了参数列表，同时改善了代码可读性；让代码更具语义性，将“点”这一概念显式地抛了出来；更容易确保X和Y的合法性，因为可以轻易地在`Point`类的构造方法中加入验证逻辑；为所有与“点”相关的行为提供了“容身之处”，否则这些行为将散落在许多与“点”打交道的方法中

随着地图引用开发的深入进行，我们发现需要几个稍加变化的“基本点”

1. 星标点，用于标注地图上的显著位置
2. 模糊点，用于标注在这片区域中的某个地方。这类点有一个以米为单位的模糊半径，会根据半径在地图上显式一个彩色的圆圈